In [9]:
import itertools
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import transforms
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

PROJECT_DIR = os.path.dirname(os.getcwd())
if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)

from chord_recognition.cache import HDF5Cache
from chord_recognition.models.deep_harmony import deep_harmony
from chord_recognition.dataset import ChromaDataset, prepare_datasource, undersample_dataset, StackedFrameDataset
from chord_recognition.utils import Rescale, one_hot
from chord_recognition.train import get_weighted_random_sampler, Solver
from chord_recognition.ann_utils import convert_annotation_matrix, get_chord_labels
from chord_recognition.evaluate import plot_confusion_matrix
from chord_recognition.predict import forward


torch.manual_seed(2020)
RANDOM_STATE = 42

%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 5)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# Exp6
window_size = 8192
hop_length = 4410
context_size = 7
T = 20
S = 8

#ds = prepare_datasource(('beatles', 'zweieck', 'queen', 'robbie_williams'))
ds = prepare_datasource(('beatles',))[:30]
cache = HDF5Cache(os.path.join(PROJECT_DIR, 'spectrogram_ann_cache.hdf5'))
dataset = StackedFrameDataset(
    datasource=ds,
    T=T,
    S=S,
    context_size=context_size,
    window_size=window_size,
    hop_length=hop_length,
    cache=cache)

In [11]:
# Split dataset into train/val
indices = np.arange(len(dataset))
idx_train, idx_val = train_test_split(
    indices, test_size=0.2, random_state=RANDOM_STATE)

In [12]:
# Calculate TRAIN_MEAN, TRAIN_STD
X_train_temp = np.hstack([xi.reshape(xi.shape[1], xi.shape[0] * xi.shape[2])
                          for xi,_ in dataset[idx_train]])
TRAIN_MEAN = X_train_temp.mean(axis=1).reshape(-1, 1)
TRAIN_STD = X_train_temp.std(axis=1).reshape(-1, 1)

del X_train_temp

In [13]:
train_data = Subset(dataset, idx_train)
val_data = Subset(dataset, idx_val)
train_data.dataset.transform = transforms.Compose([
    # Rescale inputs to have a mean of 0 and std of 1
    # It must speed up the convergence
    Rescale(TRAIN_MEAN, TRAIN_STD),
])
val_data.dataset.transform = transforms.Compose([
    # Rescale inputs to have a mean of 0 and std of 1
    # It must speed up the convergence
    Rescale(TRAIN_MEAN, TRAIN_STD),
])

In [14]:
batch_size = 8

# Split dataset into train/val datasets
loader_train = DataLoader(
    dataset=train_data,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
    num_workers=0)
loader_val = DataLoader(
    dataset=val_data,
    num_workers=0,
    batch_size=batch_size)

dataloaders = {
    "train": loader_train,
    "val": loader_val
}

In [18]:
model = deep_harmony()
if torch.cuda.is_available():
    model.cuda()

learning_rate = 1e-3
epochs=10
weight_decay = 1e-4
optimizer = optim.Adam(
    model.parameters(),
    lr=learning_rate,
    betas=(0.9, 0.999),
    weight_decay=weight_decay)

solver = Solver(
    model=model,
    optimizer=optimizer,
    dataloaders=dataloaders,
    trained_model_name="deep_auditory_v2_exp6_1.pth",
    epochs=epochs)
solver.train()

KeyboardInterrupt: 

In [ ]:
device = torch.device('cpu')

model = deep_auditory_v2(pretrained=True, model_name='deep_auditory_v2_exp6.pth')
if torch.cuda.is_available():
    device = torch.device('cuda')
    model.cuda()
model.eval()

In [ ]:
val_loader = (i for i,_ in loader_val)
y_hat_matrix = forward(model, val_loader, device, 25)

In [ ]:
y_hat_matrix = y_hat_matrix.cpu().data.numpy()

In [ ]:
y_pred = np.argmax(y_hat_matrix, 1)
y_pred = one_hot(y_pred, 25)
y_pred = convert_annotation_matrix(y_pred)

In [ ]:
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['figure.dpi'] = 100

y_matrix = one_hot(y_val, 25)
y_true = convert_annotation_matrix(y_matrix)

labels = get_chord_labels(nonchord=True)
cm = confusion_matrix(y_true, y_pred, labels=labels)
print(cm)

plot_confusion_matrix(cm, labels, fontsize=6)